# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
df = pd.read_csv(url, index_col=0)
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
df.shape

(6000, 61)

In [3]:
X = df.drop(['url', 'Popular'], axis=1)
y = df['Popular']

In [4]:
y.mean()

0.5

In [5]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [6]:
#Estandarización de la base x
from sklearn.preprocessing import StandardScaler
sc= StandardScaler() #definición de la función de estandarización
sc.fit(X_train) # defino con que media y varianza se ajustara los datos
x_train_std = sc.transform(X_train) # Definición de la variable estandarizada
x_test_std = sc.transform(X_test)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [7]:
#Logistic Regresion:
from sklearn.linear_model import LogisticRegression #Importando la librería de regresión Lineal con Sklearn
lr= LogisticRegression(solver='liblinear',C=1e9, random_state=1)# Definiendo la función "Lr" con sus parámetros
lr.fit(x_train_std,y_train)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=1,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [8]:
y_pred =lr.predict(x_test_std) # Predicción de los valores para X_test
from sklearn import metrics
print('f1 test:%2f'%metrics.f1_score(y_pred, y_test)) 
print('Accuracy test:%2f'% metrics.accuracy_score(y_pred, y_test))

f1 test:0.632373
Accuracy test:0.642667


In [9]:
#Decision tree
from sklearn.tree import DecisionTreeClassifier
treeclf = DecisionTreeClassifier(max_depth=None, random_state=123)
treeclf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [10]:
y_pred1 =treeclf.predict(X_test) # Predicción de los valores para X_test
from sklearn import metrics
print('f1 test:%2f'%metrics.f1_score(y_pred1, y_test)) 
print('Accuracy test:%2f'% metrics.accuracy_score(y_pred1, y_test))

f1 test:0.542800
Accuracy test:0.540667


# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [11]:
# instruct BaggingClassifier to use DecisionClassifier as the "base estimator, Max_depth=None"
from sklearn.ensemble import BaggingClassifier
bagreg1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=None, random_state=123), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)

In [12]:
bagreg1.fit(X_train, y_train)
y_pred1 = bagreg1.predict(X_test)

In [13]:
print('f1 test:%2f'%metrics.f1_score(y_pred1, y_test)) 
print('Accuracy test:%2f'% metrics.accuracy_score(y_pred1, y_test))

f1 test:0.635753
Accuracy test:0.638667


In [14]:
# instruct BaggingClassifier to use DecisionClassifier as the "base estimator, Max_depth=2"
from sklearn.ensemble import BaggingClassifier
bagreg2 = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=2, random_state=123), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)

In [15]:
bagreg2.fit(X_train, y_train)
y_pred2 = bagreg2.predict(X_test)
print('f1 test:%2f'%metrics.f1_score(y_pred2, y_test)) 
print('Accuracy test:%2f'% metrics.accuracy_score(y_pred2, y_test))

f1 test:0.643478
Accuracy test:0.644667


In [16]:
# instruct Logit Classifier to use DecisionClassifier as the "base estimator, 
from sklearn.ensemble import BaggingClassifier
bagreg3 = BaggingClassifier(base_estimator=LogisticRegression(solver='liblinear',C=1e9, random_state=1), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)

In [17]:
bagreg3.fit(x_train_std, y_train)
y_pred3 = bagreg3.predict(x_test_std)
print('f1 test:%2f'%metrics.f1_score(y_pred3, y_test)) 
print('Accuracy test:%2f'% metrics.accuracy_score(y_pred3, y_test))

f1 test:0.631001
Accuracy test:0.641333


# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [18]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[('lr', bagreg3), ('dt2', bagreg2), ('dtn', bagreg1)],voting='hard')
voting_clf.fit(x_train_std, y_train)

VotingClassifier(estimators=[('lr', BaggingClassifier(base_estimator=LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=1,
          solver='liblinear', tol=0.0001, verbos... n_estimators=100, n_jobs=1, oob_score=True,
         random_state=1, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [19]:
y_pred4 = voting_clf.predict(x_test_std)

C:\Users\A0685215\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [20]:
print('f1 test:%2f'%metrics.f1_score(y_pred4, y_test)) 
print('Accuracy test:%2f'% metrics.accuracy_score(y_pred4, y_test))

f1 test:0.644159
Accuracy test:0.648667


# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [21]:
# instruct BaggingClassifier to use DecisionClassifier as the "base estimator, Max_depth=None"
from sklearn.ensemble import BaggingClassifier
bagreg1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=None, random_state=123), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)
bagreg1.fit(X_train, y_train)
y_pred1 = bagreg1.predict_proba(X_test)

In [22]:
y_pred1

array([[0.17, 0.83],
       [0.35, 0.65],
       [0.46, 0.54],
       ...,
       [0.45, 0.55],
       [0.53, 0.47],
       [0.66, 0.34]])

In [23]:
import numpy as np 
import pylab as pl
probabilidad= pl.frange(0.0,1,0.1)
for i in probabilidad:
    y_pred1 = (bagreg1.predict_proba(X_test)[:,1]>=i).astype(bool)
    y_pred_dt1=np.where(y_pred1==True,1,0)
    print('f1 test:%2f'% i,metrics.f1_score(y_pred1, y_test)) 
    print('Accuracy test:%2f'% i,metrics.accuracy_score(y_pred1, y_test))

C:\Users\A0685215\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: MatplotlibDeprecationWarning: numpy.arange
  This is separate from the ipykernel package so we can avoid doing imports until


f1 test:0.000000 0.6702127659574468
Accuracy test:0.000000 0.504
f1 test:0.100000 0.6729138777331549
Accuracy test:0.100000 0.5113333333333333
f1 test:0.200000 0.6866635902168897
Accuracy test:0.200000 0.5473333333333333
f1 test:0.300000 0.700246913580247
Accuracy test:0.300000 0.5953333333333334
f1 test:0.400000 0.6956521739130435
Accuracy test:0.400000 0.6266666666666667
f1 test:0.500000 0.6359447004608294
Accuracy test:0.500000 0.6313333333333333
f1 test:0.600000 0.5206402695871946
Accuracy test:0.600000 0.6206666666666667
f1 test:0.700000 0.3267429760665973
Accuracy test:0.700000 0.5686666666666667
f1 test:0.800000 0.1234866828087167
Accuracy test:0.800000 0.5173333333333333
f1 test:0.900000 0.015706806282722512
Accuracy test:0.900000 0.49866666666666665
f1 test:1.000000 0.0
Accuracy test:1.000000 0.496


C:\Users\A0685215\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [24]:
# Predicción Probabilidad Modelo 2 : Decisiontree Max_Depth=2 
y_pred2 = bagreg2.predict_proba(X_test)
y_pred2

array([[0.29674311, 0.70325689],
       [0.36550694, 0.63449306],
       [0.36212404, 0.63787596],
       ...,
       [0.32010389, 0.67989611],
       [0.59688154, 0.40311846],
       [0.68177572, 0.31822428]])

In [25]:
import numpy as np 
y_pred2 = (bagreg2.predict_proba(X_test)[:,1]>=0.4).astype(bool)
y_pred_dt2=np.where(y_pred2==True,1,0)
import pylab as pl
probabilidad= pl.frange(0.0,1,0.1)
for i in probabilidad:
    y_pred2 = (bagreg2.predict_proba(X_test)[:,1]>=i).astype(bool)
    y_pred_dt2=np.where(y_pred2==True,1,0)
    print('f1 test:%2f'% i,metrics.f1_score(y_pred2, y_test)) 
    print('Accuracy test:%2f'% i,metrics.accuracy_score(y_pred2, y_test))

C:\Users\A0685215\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: MatplotlibDeprecationWarning: numpy.arange
  """


f1 test:0.000000 0.6702127659574468
Accuracy test:0.000000 0.504
f1 test:0.100000 0.6702127659574468
Accuracy test:0.100000 0.504
f1 test:0.200000 0.6702127659574468
Accuracy test:0.200000 0.504
f1 test:0.300000 0.675336322869955
Accuracy test:0.300000 0.5173333333333333
f1 test:0.400000 0.702176403207331
Accuracy test:0.400000 0.6533333333333333
f1 test:0.500000 0.6434782608695653
Accuracy test:0.500000 0.6446666666666667
f1 test:0.600000 0.5701078582434514
Accuracy test:0.600000 0.628
f1 test:0.700000 0.180327868852459
Accuracy test:0.700000 0.5333333333333333
f1 test:0.800000 0.0
Accuracy test:0.800000 0.496
f1 test:0.900000 0.0
Accuracy test:0.900000 0.496
f1 test:1.000000 0.0
Accuracy test:1.000000 0.496


C:\Users\A0685215\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [26]:
# instruct Logit Classifier to use DecisionClassifier as the "base estimator, 
from sklearn.ensemble import BaggingClassifier
bagreg3 = BaggingClassifier(base_estimator=LogisticRegression(solver='liblinear',C=1e9, random_state=1), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)
bagreg3.fit(x_train_std, y_train)
# Predicción Probabilidad Modelo 3 : Regresión Lineal
y_pred3 = bagreg3.predict_proba(x_test_std)
y_pred3

array([[0.34225656, 0.65774344],
       [0.34943375, 0.65056625],
       [0.36427564, 0.63572436],
       ...,
       [0.43129733, 0.56870267],
       [0.73789391, 0.26210609],
       [0.68886952, 0.31113048]])

In [27]:
# Maximización de F1- Scor modelo Logit
import numpy as np 
import pylab as pl
probabilidad= pl.frange(0.0,1,0.1)
for i in probabilidad:
    y_pred3 = (bagreg3.predict_proba(x_test_std)[:,1]>=i).astype(bool)
    y_pred_lr=np.where(y_pred3==True,1,0)
    print('f1 test:%2f'% i,metrics.f1_score(y_pred3, y_test)) 
    print('Accuracy test:%2f'% i,metrics.accuracy_score(y_pred3, y_test))

C:\Users\A0685215\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: MatplotlibDeprecationWarning: numpy.arange
  after removing the cwd from sys.path.


f1 test:0.000000 0.6702127659574468
Accuracy test:0.000000 0.504
f1 test:0.100000 0.6702127659574468
Accuracy test:0.100000 0.504
f1 test:0.200000 0.6811397557666216
Accuracy test:0.200000 0.53
f1 test:0.300000 0.6977205153617442
Accuracy test:0.300000 0.5933333333333334
f1 test:0.400000 0.7029192902117917
Accuracy test:0.400000 0.654
f1 test:0.500000 0.6310013717421126
Accuracy test:0.500000 0.6413333333333333
f1 test:0.600000 0.5172124265323258
Accuracy test:0.600000 0.6166666666666667
f1 test:0.700000 0.35294117647058826
Accuracy test:0.700000 0.582
f1 test:0.800000 0.13349514563106793
Accuracy test:0.800000 0.524
f1 test:0.900000 0.025940337224383915
Accuracy test:0.900000 0.49933333333333335
f1 test:1.000000 0.0
Accuracy test:1.000000 0.496


C:\Users\A0685215\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


La probabilidad que maximiza el modelo de clasificación Logit es 0.4

# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [28]:
n_estimators = 100
# set a seed for reproducibility
np.random.seed(123)

n_samples = X_train.shape[0]

# create bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(n_estimators)]


In [29]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[('lr', bagreg3), ('dt2', bagreg2), ('dtn', bagreg1)],voting='hard')
voting_clf.fit(x_train_std, y_train)

VotingClassifier(estimators=[('lr', BaggingClassifier(base_estimator=LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=1,
          solver='liblinear', tol=0.0001, verbos... n_estimators=100, n_jobs=1, oob_score=True,
         random_state=1, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [31]:
errors = np.zeros(voting_clf.n_estimators)
y_pred_all_ = np.zeros((X_test.shape[0], voting_clf.n_estimators))

AttributeError: 'VotingClassifier' object has no attribute 'n_estimators'

In [ ]:
errors = np.zeros(clf.n_estimators)
y_pred_all_ = np.zeros((X_test.shape[0], clf.n_estimators))

for i in range(clf.n_estimators):
    oob_sample = ~clf.estimators_samples_[i]
    y_pred_ = clf.estimators_[i].predict(X_train.values[oob_sample])
    errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
    y_pred_all_[:, i] = clf.estimators_[i].predict(X_test)
    
alpha = (1 - errors) / (1 - errors).sum()
y_pred = (np.sum(y_pred_all_ * alpha, axis=1) >= 0.5).astype(np.int)

# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score